In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
import scipy.spatial.distance as ssd
import random
import pickle
import time

In [2]:
import utils
import treerep
import hccfit
import rootedtreefit

In [3]:
def job(G, D_original, l_steps = 10, n_seed = -1, n_edges = 500, u_delta = 0.1, method = 'hcc'):
    tree = G.copy()
    start_time = time.time()
    N = D_original.shape[0]
#     If n_seed = -1, then we won't fix the seed. If not, 
    if(n_seed >= 0):
        np.random.seed(n_seed)
    edge_list = []
    while(len(edge_list) < n_edges):
        pi = np.random.permutation(N)
        i, j = pi[0], pi[1]
        if ((i,j) not in edge_list) and ((j,i) not in edge_list) and (D_original[i,j] > 2.0):
            edge_list.append((i,j))
    error_tape = []
    for idx in range(n_edges):
        i = edge_list[idx][0]
        j = edge_list[idx][1]
        tree.add_edge(i, j, weight = D_original[i,j] - 2 * u_delta)
        if(idx%l_steps == l_steps - 1):
            d = np.zeros((N,N))
            p = dict(nx.shortest_path_length(tree, method = 'bellman-ford', weight = 'weight'))
#             end = time.time()
            #     print("Dijkstra takes ", end - start, " secs")
            for x in range(N):
                for y in range(N):
                    d[x][y] = p[x][y]
#             print("d computed")
            error = []
            if(method == 'TR'):
                for tr_seed in range(50):
                    np.random.seed(tr_seed)
                    T = treerep.TreeRep(d)
                    T.learn_tree()
                    for e in T.G.edges():
                        if(T.G[e[0]][e[1]]['weight'] < 0):
                            # print(e[0], e[1])
                            T.G[e[0]][e[1]]['weight'] = 0
                        # T.G[e[0]][e[1]]['weight'] = self.W[e[0],e[1]]
                    d_T = np.zeros((N,N))
                    start = time.time()
                    p = dict(nx.shortest_path_length(T.G, method = 'bellman-ford', weight = 'weight'))
                    end = time.time()
                    #     print("Dijkstra takes ", end - start, " secs")
                    for x in range(N):
                        for y in range(N):
                            d_T[x][y] = p[x][y]
                    error.append(np.sum(abs(d_T - d)) / (N*(N-1)))
            elif(method == 'hcc'):
                RT = rootedtreefit.RootedTreeFit(d)
                RT.fit_treeM(pivot_idx = 0, method = 'hcc')
                d_T = RT.d_T
                error.append(np.sum(abs(d_T - d)) / (N*(N-1)))
            elif(method == 'gromov'):
                RT = rootedtreefit.RootedTreeFit(d)
                RT.fit_treeM(pivot_idx = 0, method = 'gromov')
                d_T = RT.d_T
                error.append(np.sum(abs(d_T - d)) / (N*(N-1)))
            elif(method == 'average'):
                RT = rootedtreefit.RootedTreeFit(d)
                RT.fit_treeM(pivot_idx = 0, method = 'average')
                d_T = RT.d_T
                error.append(np.sum(abs(d_T - d)) / (N*(N-1)))
            elif(method == 'complete'):
                RT = rootedtreefit.RootedTreeFit(d)
                RT.fit_treeM(pivot_idx = 0, method = 'complete')
                d_T = RT.d_T
                error.append(np.sum(abs(d_T - d)) / (N*(N-1)))
            elif(method == 'NJ'):
                NJtree = utils.NeighborJoin(d)
                for e in NJtree.edges():
                    if(NJtree[e[0]][e[1]]['weight'] < 0):
                        # print(e[0], e[1])
                        NJtree[e[0]][e[1]]['weight'] = 0
#                         print('negative edge has been found')
#                 print("Time takes ", time.time() - start)
                length = dict(nx.all_pairs_dijkstra_path_length(NJtree))
                n_tree = len(NJtree)
                d_T = np.zeros((N,N))
                for i in range(N):
                    for j in range(N):
                        d_T[i][j] = length[i][j]
                error.append(np.sum(abs(d_T - d)) / (N*(N-1)))
            error_tape.append(error)
    return error_tape

In [4]:
# Inputs we have used
b_tree_8_3 = nx.balanced_tree(r=8, h=3)
b_tree_5_4 = nx.balanced_tree(r=5, h=4)
b_tree_3_5 = nx.balanced_tree(r=3, h=5)
b_tree_2_8 = nx.balanced_tree(r=2, h=8)

In [5]:
import csv
# For disease, use follows. (We have removed header line)
# opening the CSV file
with open('dataset/disease_lp/disease_lp.edges.csv', mode ='r') as file:
    csvFile = csv.reader(file)
    disease_lp_edges = []
    n_lines = 0
    # displaying the contents of the CSV file
    for lines in csvFile:
#         print(lines)
        disease_lp_edges.append(lines)
        n_lines += 1
disease_lp_G = nx.Graph()
for e in disease_lp_edges:
    if e[0] not in disease_lp_G.nodes:
        disease_lp_G.add_node(e[0])
    if e[1] not in disease_lp_G.nodes:
        disease_lp_G.add_node(e[0])
    disease_lp_G.add_edge(e[0], e[1])
disease_lp_G = nx.convert_node_labels_to_integers(disease_lp_G)
start = time.time()
D_disease_lp = nx.floyd_warshall_numpy(disease_lp_G)
end = time.time()
# print("Time: ", end - start)

In [6]:
D = nx.floyd_warshall_numpy(b_tree_8_3)
output = job(b_tree_8_3, D, l_steps = 100, n_seed = -1, n_edges = 500, u_delta = 0.1, method = 'hcc')
print(output)
# output = job(disease_lp_G, D_disease_lp, l_steps = 100, n_seed = -1, n_edges = 500, u_delta = 0.1, method = 'hcc')

[[0.0010537407797681778], [0.0023767708699215565], [0.0029680365296803676], [0.004025289778714439], [0.00543964406978106]]
